In [1]:
import os
from whoosh.index import open_dir
from whoosh.index import create_in
from whoosh.fields import *

from jieba.analyse import ChineseAnalyzer
from whoosh.qparser import QueryParser
from sagas.ofbiz.resources import ResourceDigester, read_resource

analyzer = ChineseAnalyzer()
schema = Schema(en=TEXT(stored=True),
    fr=TEXT(stored=True),
    key=ID(stored=True),
    zh=TEXT(stored=True, analyzer=analyzer))

In [2]:
def index_resource(resource):
    ## rewrite mode
    out_dir='out/external_refs'
    if not os.path.exists(out_dir):
        os.mkdir(out_dir)
    idx = create_in(out_dir, schema)
    writer = idx.writer()

    for key, prop in resource.properties.items():
        writer.add_document(
            key=key,
            en=prop.values['en'],
            zh=prop.values['zh'],
            fr=prop.values['fr']
        )

    writer.commit()

In [2]:
import protobuf_utils as pu
from simple_pb2 import MyObj, Foo
obj=MyObj(name='hello')
print(obj)
pu.write_proto_to(obj, 'data/resources/tests_obj.data')

name: "hello"



In [5]:
from sagas.ofbiz.entities import OfEntity as e, oc
import sagas.ofbiz.entities as ee

product=e().refProduct('GZ-2002')
ee.format(product)

+----------------------------+------------------------------------+
| name                       | value                              |
|----------------------------+------------------------------------|
| originGeoId                |                                    |
| detailScreen               |                                    |
| createdStamp               | 2019-03-03 22:33:37.435            |
| requireInventory           |                                    |
| quantityUomId              |                                    |
| mediumImageUrl             |                                    |
| productName                | Square Gizmo                       |
| introductionDate           |                                    |
| shippingHeight             |                                    |
| originalImageUrl           |                                    |
| inShippingBox              |                                    |
| detailImageUrl             |                  

In [6]:
e('meta').ProductType

,internal,name,primary,type,field type
0,,productTypeId,*,id,id
1,,parentTypeId,,id,id
2,,isPhysical,,indicator,indicator
3,,isDigital,,indicator,indicator
4,,hasTable,,indicator,indicator
5,,description,,description,description
6,*,lastUpdatedStamp,,date-time,date-time
7,*,lastUpdatedTxStamp,,date-time,date-time
8,*,createdStamp,,date-time,date-time
9,*,createdTxStamp,,date-time,date-time


In [25]:
# find(String entityName, EntityCondition whereEntityCondition, 
# EntityCondition havingEntityCondition, Set<String> fieldsToSelect, 
# List<String> orderBy, EntityFindOptions findOptions) 
iter=oc.delegator.find('ProductType', None, None, None, None, None)
# while iter.hasNext():
#     val=iter.next()
#     print(val.get('productTypeId'), val.get('description'))

# For performance reasons do not use the EntityListIterator.hasNext() method, 
# just call next() until it returns null; see JavaDoc comments in the 
# EntityListIterator class for details and an example
val=iter.next()
while val is not None:
    print(val.get('productTypeId'), val.get('description'))
    val=iter.next()

AGGREGATED Configurable Good
AGGREGATEDSERV_CONF Configurable Service Configuration
AGGREGATED_CONF Configurable Good Configuration
AGGREGATED_SERVICE Configurable Service using inventory
ASSET_USAGE Fixed Asset Usage
ASSET_USAGE_OUT_IN Fixed Asset Usage For Rental of an asset which is shipped from and returned to inventory
DIGITAL_GOOD Digital Good
FINDIG_GOOD Finished/Digital Good
FINISHED_GOOD Finished Good
GOOD Good
MARKETING_PKG Marketing Package
MARKETING_PKG_AUTO Marketing Package: Auto Manufactured
MARKETING_PKG_PICK Marketing Package: Pick Assembly
RAW_MATERIAL Raw Material
SERVICE Service
SERVICE_PRODUCT Service a product using inventory
SUBASSEMBLY Subassembly
Test_type None
Test_type_11 None
Test_type_111 None
Test_type_112 None
Test_type_113 None
Test_type_114 None
WIP Work In Process


In [18]:
from sagas.util.name_util import to_global_id, from_global_id
entity='ProductType'
model=oc.delegator.getModelEntity(entity)
pks=[]
for fld in model.getPkFieldNames():
    pks.append(fld)
print(pks)
gid=to_global_id(entity, 'FINDIG_GOOD')
print(gid)

params=oc.jmap()
t,idstring=from_global_id(gid)
ids=idstring.split('♡')
print(ids)
mapped=zip(pks, ids)
ctx=list(mapped) 
print(ctx)
# print(ctx[0][0])
for el in ctx:
    params.put(el[0], el[1])
result = oc.delegator.findOne(entity, params, True)
ee.format(result)

['productTypeId']
UHJvZHVjdFR5cGU6RklORElHX0dPT0Q=
['FINDIG_GOOD']
[('productTypeId', 'FINDIG_GOOD')]
+--------------------+-------------------------+
| name               | value                   |
|--------------------+-------------------------|
| lastUpdatedStamp   | 2019-03-03 22:33:14.056 |
| isPhysical         | Y                       |
| parentTypeId       | GOOD                    |
| isDigital          | Y                       |
| hasTable           | N                       |
| createdTxStamp     | 2019-03-03 22:33:13.489 |
| createdStamp       | 2019-03-03 22:33:14.056 |
| description        | Finished/Digital Good   |
| lastUpdatedTxStamp | 2019-03-03 22:33:13.489 |
| productTypeId      | FINDIG_GOOD             |
+--------------------+-------------------------+


In [19]:
import sagas.ofbiz.entity_global_ref as gr
ent_ref=gr.EntityGlobalRef('ProductType')
ent_ref.fill_records('productTypeId', 'description')

[('UHJvZHVjdFR5cGU6QUdHUkVHQVRFRA==', 'AGGREGATED', 'Configurable Good'),
 ('UHJvZHVjdFR5cGU6QUdHUkVHQVRFRFNFUlZfQ09ORg==',
  'AGGREGATEDSERV_CONF',
  'Configurable Service Configuration'),
 ('UHJvZHVjdFR5cGU6QUdHUkVHQVRFRF9DT05G',
  'AGGREGATED_CONF',
  'Configurable Good Configuration'),
 ('UHJvZHVjdFR5cGU6QUdHUkVHQVRFRF9TRVJWSUNF',
  'AGGREGATED_SERVICE',
  'Configurable Service using inventory'),
 ('UHJvZHVjdFR5cGU6QVNTRVRfVVNBR0U=', 'ASSET_USAGE', 'Fixed Asset Usage'),
 ('UHJvZHVjdFR5cGU6QVNTRVRfVVNBR0VfT1VUX0lO',
  'ASSET_USAGE_OUT_IN',
  'Fixed Asset Usage For Rental of an asset which is shipped from and returned to inventory'),
 ('UHJvZHVjdFR5cGU6RElHSVRBTF9HT09E', 'DIGITAL_GOOD', 'Digital Good'),
 ('UHJvZHVjdFR5cGU6RklORElHX0dPT0Q=', 'FINDIG_GOOD', 'Finished/Digital Good'),
 ('UHJvZHVjdFR5cGU6RklOSVNIRURfR09PRA==', 'FINISHED_GOOD', 'Finished Good'),
 ('UHJvZHVjdFR5cGU6R09PRA==', 'GOOD', 'Good'),
 ('UHJvZHVjdFR5cGU6TUFSS0VUSU5HX1BLRw==',
  'MARKETING_PKG',
  'Marketing Package'

In [27]:
from values_pb2 import ExternalLinks, ExternalLink
rs=ent_ref.fill_records('productTypeId', 'description')
links=[]
for rec in rs:
    link=ExternalLink(gid=rec[0], name=rec[1], description=rec[2])
    links.append(link)
links_rs=ExternalLinks(links=links)
print(len(links_rs.links))
print(links_rs)

24
links {
  gid: "UHJvZHVjdFR5cGU6QUdHUkVHQVRFRA=="
  name: "AGGREGATED"
  description: "Configurable Good"
}
links {
  gid: "UHJvZHVjdFR5cGU6QUdHUkVHQVRFRFNFUlZfQ09ORg=="
  name: "AGGREGATEDSERV_CONF"
  description: "Configurable Service Configuration"
}
links {
  gid: "UHJvZHVjdFR5cGU6QUdHUkVHQVRFRF9DT05G"
  name: "AGGREGATED_CONF"
  description: "Configurable Good Configuration"
}
links {
  gid: "UHJvZHVjdFR5cGU6QUdHUkVHQVRFRF9TRVJWSUNF"
  name: "AGGREGATED_SERVICE"
  description: "Configurable Service using inventory"
}
links {
  gid: "UHJvZHVjdFR5cGU6QVNTRVRfVVNBR0U="
  name: "ASSET_USAGE"
  description: "Fixed Asset Usage"
}
links {
  gid: "UHJvZHVjdFR5cGU6QVNTRVRfVVNBR0VfT1VUX0lO"
  name: "ASSET_USAGE_OUT_IN"
  description: "Fixed Asset Usage For Rental of an asset which is shipped from and returned to inventory"
}
links {
  gid: "UHJvZHVjdFR5cGU6RElHSVRBTF9HT09E"
  name: "DIGITAL_GOOD"
  description: "Digital Good"
}
links {
  gid: "UHJvZHVjdFR5cGU6RklORElHX0dPT0Q="
  name: "F

In [22]:
import protobuf_utils as pu
pu.write_proto_to(links_rs, 'data/resources/rs_product_type.data')

In [28]:
desc_fld=['id','name']
isinstance(desc_fld, list)

True